<a href="https://colab.research.google.com/github/CarlOsito16/cpnEE/blob/main/cpnEE_new.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch transformers accelerate>=0.20.1 datasets evaluate

In [5]:
!rm -rf /content/800_hf_data

In [6]:
!unzip 800_hf_data.zip

Archive:  800_hf_data.zip
   creating: 800_hf_data/
  inflating: 800_hf_data/dataset_dict.json  
   creating: 800_hf_data/test/
   creating: 800_hf_data/train/
   creating: 800_hf_data/eval/
  inflating: 800_hf_data/test/state.json  
  inflating: 800_hf_data/test/dataset_info.json  
  inflating: 800_hf_data/test/data-00000-of-00001.arrow  
  inflating: 800_hf_data/train/state.json  
  inflating: 800_hf_data/train/dataset_info.json  
  inflating: 800_hf_data/train/data-00000-of-00001.arrow  
  inflating: 800_hf_data/eval/state.json  
  inflating: 800_hf_data/eval/dataset_info.json  
  inflating: 800_hf_data/eval/data-00000-of-00001.arrow  


In [1]:
from datasets import load_from_disk
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F


from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
import numpy as np

In [2]:
complete_dataset_dict  = load_from_disk("800_hf_data")
complete_dataset_dict

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'Question', 'RawComment', 'OriginalLabel', 'Label', 'Sentiment', 'cleanComment', 'translated', '__index_level_0__'],
        num_rows: 640
    })
    eval: Dataset({
        features: ['Unnamed: 0', 'Question', 'RawComment', 'OriginalLabel', 'Label', 'Sentiment', 'cleanComment', 'translated', '__index_level_0__'],
        num_rows: 80
    })
    test: Dataset({
        features: ['Unnamed: 0', 'Question', 'RawComment', 'OriginalLabel', 'Label', 'Sentiment', 'cleanComment', 'translated', '__index_level_0__'],
        num_rows: 80
    })
})

In [3]:
complete_dataset_dict['train'].features

{'Unnamed: 0': Value(dtype='int64', id=None),
 'Question': Value(dtype='string', id=None),
 'RawComment': Value(dtype='string', id=None),
 'OriginalLabel': Value(dtype='string', id=None),
 'Label': Value(dtype='string', id=None),
 'Sentiment': ClassLabel(names=['Negative', 'Positive'], id=None),
 'cleanComment': Value(dtype='string', id=None),
 'translated': Value(dtype='string', id=None),
 '__index_level_0__': Value(dtype='int64', id=None)}

In [4]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/MiniLM-L12-H384-uncased")

In [5]:
# define tokenizer funciton to be used with dataset.Dataset
def tokenize_dataset(examples):
    return tokenizer(examples['translated'], #here we must specify the name of the input text column
                     max_length = 64,
                     truncation = True,
                     padding = "max_length")



In [6]:
# The beauty of using datasets.Dataset is that we can use .map with the `tokenize_dataset` function directly
complete_dataset_dict = complete_dataset_dict.map(tokenize_dataset, batched= True)
complete_dataset_dict

Map:   0%|          | 0/80 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'Question', 'RawComment', 'OriginalLabel', 'Label', 'Sentiment', 'cleanComment', 'translated', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 640
    })
    eval: Dataset({
        features: ['Unnamed: 0', 'Question', 'RawComment', 'OriginalLabel', 'Label', 'Sentiment', 'cleanComment', 'translated', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 80
    })
    test: Dataset({
        features: ['Unnamed: 0', 'Question', 'RawComment', 'OriginalLabel', 'Label', 'Sentiment', 'cleanComment', 'translated', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 80
    })
})

In [7]:
complete_dataset_dict['train'].features

{'Unnamed: 0': Value(dtype='int64', id=None),
 'Question': Value(dtype='string', id=None),
 'RawComment': Value(dtype='string', id=None),
 'OriginalLabel': Value(dtype='string', id=None),
 'Label': Value(dtype='string', id=None),
 'Sentiment': ClassLabel(names=['Negative', 'Positive'], id=None),
 'cleanComment': Value(dtype='string', id=None),
 'translated': Value(dtype='string', id=None),
 '__index_level_0__': Value(dtype='int64', id=None),
 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None),
 'token_type_ids': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None),
 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None)}

In [8]:
complete_dataset_dict['train'].to_pandas().head()

,Unnamed: 0,Question,RawComment,OriginalLabel,Label,Sentiment,cleanComment,translated,__index_level_0__,input_ids,token_type_ids,attention_mask
0,947,When talking about your work experience in the...,ความร่วมมือในการทำงาน | ความร่วมมือคนในแผนกยัง...,ความร่วมมือในการทำงาน,Collaboration,0,ความร่วมมือคนในแผนกยังมีความเห็นแก่ตัวกันมาก,"Cooperation, people in the department are stil...",721,"[101, 6792, 1010, 2111, 1999, 1996, 2533, 2024...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, ..."
1,1980,What do you like most about working in this or...,สวัสดิการ | เวลาเจ็บไข้ได้ป่วยสามารถไปรักษาได้...,สวัสดิการ,Compensation and Benefits,1,เวลาเจ็บไข้ได้ป่วยสามารถไปรักษาได้โดยไม่ต้องเส...,"When you get sick, you can go for treatment wi...",260,"[101, 2043, 2017, 2131, 5305, 1010, 2017, 2064...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,4607,What do you like most about working in this or...,หัวหน้างานโดยตรง | หัวหน้างานและทีมงานภายในศูน...,หัวหน้างานโดยตรง,Direct Supervisor or Top Management,1,หัวหน้างานและทีมงานภายในศูนย์ให้ความร่วมมือและ...,Supervisors and team members in the center coo...,494,"[101, 22565, 1998, 2136, 2372, 1999, 1996, 241...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3,4692,When talking about your work experience in the...,หัวหน้างานโดยตรง | อยากให้มีการสื่อสารข่าวสารก...,หัวหน้างานโดยตรง,Direct Supervisor or Top Management,0,อยากให้มีการสื่อสารข่าวสารกิจกรรมต่างๆของบริษั...,I would like to communicate news and activitie...,589,"[101, 1045, 2052, 2066, 2000, 10639, 2739, 199...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,1189,When talking about your work experience in the...,ช่องทางการสื่อสาร | การสื่อสารต่างแผนกควรสื่อส...,ช่องทางการสื่อสาร,Collaboration,0,การสื่อสารต่างแผนกควรสื่อสารให้ชัดเจน,Communication from different departments shoul...,712,"[101, 4807, 2013, 2367, 7640, 2323, 2022, 4415...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, ..."


# Next step is very import to preprocess the dataset to be ready for the trainer

In [9]:
complete_dataset_dict = complete_dataset_dict.remove_columns(['Unnamed: 0', 'Question', 'RawComment', 'OriginalLabel','Label',  'cleanComment', 'translated', '__index_level_0__' ])
complete_dataset_dict = complete_dataset_dict.rename_column('Sentiment', "labels")

In [10]:
complete_dataset_dict

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 640
    })
    eval: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 80
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 80
    })
})

In [11]:
complete_dataset_dict['train'].features

{'labels': ClassLabel(names=['Negative', 'Positive'], id=None),
 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None),
 'token_type_ids': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None),
 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None)}

## Start calling the pre-trained model

In [12]:
num_labels = 2
model2 = AutoModelForSequenceClassification.from_pretrained("microsoft/MiniLM-L12-H384-uncased", num_labels=num_labels)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at microsoft/MiniLM-L12-H384-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [137]:
def compute_metrics(eval_preds):
    acc_metric = evaluate.load("accuracy")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return acc_metric.compute(predictions=predictions, references=labels)

In [13]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
def compute_metrics(p):
    pred, labels = p
    pred = np.argmax(pred, axis=-1)
    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred)
    precision = precision_score(y_true=labels, y_pred=pred)
    f1 = f1_score(y_true=labels, y_pred=pred)
    return {"accuracy": accuracy, "f1": f1}

In [15]:
batch_size = 16
output_dir = 'training_output'
logging_steps = len(complete_dataset_dict['train']) // batch_size
training_args = TrainingArguments(output_dir=output_dir,
                                  num_train_epochs=8,
                                  learning_rate=0.0001,
                                  logging_steps=logging_steps,
                                  evaluation_strategy='epoch',
                                  disable_tqdm=False)

In [16]:
trainer = Trainer(model = model2,
                  args =training_args,
                  train_dataset =complete_dataset_dict['train'],
                  eval_dataset = complete_dataset_dict['eval'],
                  tokenizer = tokenizer,
                  compute_metrics= compute_metrics)
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.497800,0.515094,0.787500,0.779221
2,0.513200,0.398399,0.862500,0.864198
3,0.336600,0.325411,0.900000,0.900000
4,0.268200,0.431132,0.862500,0.857143
5,0.278700,0.376865,0.875000,0.868421
6,0.219000,0.318618,0.900000,0.900000
7,0.196000,0.291752,0.925000,0.925000
8,0.167000,0.246987,0.937500,0.936709


TrainOutput(global_step=640, training_loss=0.32126558795571325, metrics={'train_runtime': 65.6908, 'train_samples_per_second': 77.941, 'train_steps_per_second': 9.743, 'total_flos': 42158547271680.0, 'train_loss': 0.32126558795571325, 'epoch': 8.0})

In [39]:
eval_prediction = trainer.predict(complete_dataset_dict['test'])[1]
eval_prediction

array([0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0,
       0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0,
       0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1,
       0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0])

In [40]:
np.array(complete_dataset_dict['test'].to_pandas().labels)

array([0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0,
       0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0,
       0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1,
       0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0])

In [34]:
from sklearn.metrics import confusion_matrix, classification_report

In [41]:
print(classification_report(eval_prediction,
                            np.array(complete_dataset_dict['test'].to_pandas().labels) ))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        40
           1       1.00      1.00      1.00        40

    accuracy                           1.00        80
   macro avg       1.00      1.00      1.00        80
weighted avg       1.00      1.00      1.00        80



In [69]:
import evaluate

In [70]:
accuracy_metric = evaluate.load('accuracy')
accuracy_metric = evaluate.load('accuracy')

In [144]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.691000,0.744997,0.500000
2,0.702100,0.693148,0.500000
3,0.694700,0.697766,0.500000
4,0.699000,0.707279,0.500000
5,0.697700,0.694788,0.500000
6,0.694100,0.693809,0.500000
7,0.696300,0.693219,0.500000
8,0.693300,0.693147,0.500000


TrainOutput(global_step=640, training_loss=0.7002702862024307, metrics={'train_runtime': 46.5491, 'train_samples_per_second': 109.991, 'train_steps_per_second': 13.749, 'total_flos': 42158547271680.0, 'train_loss': 0.7002702862024307, 'epoch': 8.0})

In [ ]:
model.device

device(type='cuda', index=0)

In [76]:
# Set the model to evaluation mode
model.eval()

example_sentence = "I hate working here"
# Preprocess the input text
inputs = tokenizer(example_sentence,  max_length = 64,
                     truncation = True,
                     padding = "max_length", return_tensors="pt").to('cuda') #has to specify to('cuda') to avoid the error# Get the predicted label

# Perform prediction
with torch.no_grad():
    outputs = model(**inputs)

In [77]:
outputs

SequenceClassifierOutput(loss=None, logits=tensor([[-1.4029,  1.3609]], device='cuda:0'), hidden_states=None, attentions=None)

In [78]:
# Get the predicted label
predicted_label = torch.argmax(outputs.logits).item()

In [79]:
predicted_label

1

In [145]:
trainer.train_dataset[0]

{'labels': 0,
 'input_ids': [101,
  6792,
  1010,
  2111,
  1999,
  1996,
  2533,
  2024,
  2145,
  2200,
  14337,
  1012,
  102,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'token_type_ids': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  